In [ ]:
import pandas as pd
import glob
import os

# 设置目录路径
folder_path = "/home/jesse/Projects/myprojs/MT_1/01_Data_Collection/dogecoin_comments-till_2024"

# 获取所有 CSV 文件
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 初始化一个空列表用于存放每个文件的 DataFrame
dataframes = []

# 循环读取所有文件
for file in csv_files:
    df = pd.read_csv(file)
    # 转换为 datetime 类型，强制出错为 NaT
    df['created'] = pd.to_datetime(df['created'], errors='coerce')
    dataframes.append(df)

# 合并所有 DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# 设定筛选的时间区间
start_date = pd.to_datetime("2019-07-05")
end_date = pd.to_datetime("2024-12-31")

# 筛选时间区间内的数据
filtered_df = combined_df[
    (combined_df['created'] >= start_date) &
    (combined_df['created'] <= end_date)
]

# 排序后去重（按 link 和 created）
filtered_df.sort_values(by="created", inplace=True)
filtered_df.drop_duplicates(subset=["author", "created"], keep="first", inplace=True)


# 重设索引
filtered_df.reset_index(drop=True, inplace=True)

# 保存结果到一个新 CSV 文件
output_file = os.path.join(folder_path, "dogecoin_comments_2019-07-05_to_2024-12-31_deduped.csv")
filtered_df.to_csv(output_file, index=False)

print(f"✅ 合并+筛选+去重完成，文件已保存至: {output_file}")
